In [3]:
import os
from pathlib import Path
from gurobipy import Model, GRB
from data_loader import load_prices, load_storage, load_efficiency, load_plant_capacity, load_demand, load_params
import pandas as pd

# Parameter path
current_dir = Path().resolve()
parameter_file = os.path.join(current_dir, "params.csv")
# Data Loading from data folder in the parent directory
data_folder = os.path.join(current_dir.parent, "data")
fuel_prices_file = os.path.join(data_folder, "fuel_prices.csv")           # Price in EUR/tonne (coal,oil) or EUR/m3 (gas)
storage_file = os.path.join(data_folder, "storage.csv")         # Max storage in tonnes/m3
efficiency_file = os.path.join(data_folder, "efficiency.csv")   # efficiency of MWh th to MWh el
plant_file = os.path.join(data_folder, "plant_capacity.csv")    # Plant capacity in MWh/month
demand_file = os.path.join(data_folder, "electricity_demand.csv")           # Monthly demand in MWh

# Load parameters
try:
    params_dict = load_params(parameter_file)
except Exception as e:
    raise RuntimeError(f"Error loading parameters: {e}")

start_date = params_dict.get("start_date", "2020-01")
end_date = params_dict.get("end_date", "2022-12")
supply_factor = params_dict.get("supply_factor", 0.6)
zone = params_dict.get("zone", "DK_1")
resample_method = params_dict.get("resample_method", "M")
# Get every row starting with 'fuel' as fuel types
fuels = [value for key, value in params_dict.items() if key.startswith("fuel")]

try:
    fuel_prices_df = load_prices(fuel_prices_file, start_date, end_date, resample_method= resample_method)
    storage = load_storage(storage_file)
    efficiency = load_efficiency(efficiency_file)
    plant_capacity = load_plant_capacity(plant_file)
    demand_df = load_demand(demand_file, start_date, end_date, resample_method= resample_method ,zone=zone ,supply_factor=0.6)
except Exception as e:
    raise RuntimeError(f"Error loading data: {e}")


# Timesteps
T = list(range(len(demand_df)))

# Model Initialization

m = Model("Fuel_Procurement_OneMonth")

F = [fuel for fuel in fuel_prices_df.columns if fuel in fuels]

# Decision Variables
# Decision Variables (corrected filters)
x = {(fuel, t): m.addVar(lb=0, ub=storage[fuel], name=f"buy_{fuel}_{t}")
     for fuel in F for t in T}

# y = {(fuel, t): m.addVar(lb=0, ub=plant_capacity[fuel], name=f"gen_{fuel}_{t}")
#      for fuel in F for t in T}

s = {(fuel, t): m.addVar(lb=0, ub=storage[fuel], name=f"s_{fuel}_{t}")
     for fuel in F for t in T}

# periods as Month PeriodIndex (so days_in_month works)
periods = demand_df.index.to_period('M')
 
 
y = {}
for fuel in F:
    for t in T:                       
        hours_in_month = periods[t].days_in_month * 24
        # plant_capacity[fuel] is MW -> convert to MWh/month
        monthly_capacity_mwh = plant_capacity.get(fuel, 0) * hours_in_month
        y[fuel, t] = m.addVar(lb=0, ub=monthly_capacity_mwh, name=f"use_{fuel}_{t}")


m.update()



In [4]:
# Initial Storage Levels
initial_storage = {fuel: 0 for fuel in fuel_prices_df.columns if fuel in fuels}

# Constraints

# --- Add consumption (withdrawal) variables so stored fuel can be used later ---
# w = fuel consumed in a period (units same as storage units MWh th)
w = {(fuel, t): m.addVar(lb=0, ub=storage[fuel], name=f"cons_{fuel}_{t}")
     for fuel in F for t in T}

m.update()

# Demand constraint: one per time period
for t in T:
    m.addConstr(
        sum(y[fuel, t] for fuel in F) >= int(demand_df.iloc[t][zone]),
        name=f"Demand_Constraint_{t}"
    )

# efficiency constraints
for fuel in F:
    for t in T:
        m.addConstr(
            y[fuel, t] == efficiency[fuel] * w[fuel, t],
            name=f"Gen_from_Consumption_{fuel}_{t}"
        )

m.update()

# State-of-energy (SOE) constraints: storage evolves with buys (x) and withdrawals (w)
for fuel in F:
    print(f"Setting up SOE constraints for {fuel}")
    for t in T:
        print(f"  Time period {t}")
        if t == 0:
            m.addConstr(
                s[fuel, t] == initial_storage[fuel] + x[fuel, t] - w[fuel, t],
                name=f"SOE_Def_{fuel}_{t}"
            )
        else:
            m.addConstr(
                s[fuel, t] == s[fuel, t-1] + x[fuel, t] - w[fuel, t],
                name=f"SOE_Def_{fuel}_{t}"
            )

# Storage limits (explicit upper bound)
for fuel in F:
    for t in T:
        m.addConstr(
            s[fuel, t] <= storage[fuel],
            name=f"SOE_Limit_{fuel}_{t}"
        )

# Cyclic SOE (start == end)
t0 = 0
tlast = T[-1]   # last period
for fuel in F:
    m.addConstr(
        s[fuel, t0] == s[fuel, tlast],
        name=f"SOE_Cycle_{fuel}"
    )

# Objective
m.setObjective(
    sum(fuel_prices_df.iloc[t][fuel] * x[fuel, t] for fuel in F for t in T),
    GRB.MINIMIZE
)
m.update()

# Optimize
m.optimize()

# print results
if m.status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    # Purchased and generated per fuel and period
    for (fuel, t) in sorted(x.keys()):
        print(f"Purchased {x[fuel, t].X:.4f} units of {fuel} at period {t}")
    for (fuel, t) in sorted(y.keys()):
        print(f"Generated {y[fuel, t].X:.4f} MWh from {fuel} at period {t}")
    for (fuel, t) in sorted(s.keys()):
        print(f"Stored {s[fuel, t].X:.4f} units of {fuel} at period {t}")

    # Cost per fuel aggregated across periods
    total_cost = 0.0
    for fuel in F:
        cost_f = sum(fuel_prices_df.iloc[t][fuel] * x[fuel, t].X for t in T)
        print(f"Cost for {fuel}: EUR {cost_f:.2f}")
        total_cost += cost_f

    print(f"Total Cost: EUR {total_cost:.2f}")
else:
    print("No optimal solution found.")

Setting up SOE constraints for coal
  Time period 0
  Time period 1
  Time period 2
  Time period 3
  Time period 4
  Time period 5
  Time period 6
  Time period 7
  Time period 8
  Time period 9
  Time period 10
  Time period 11
  Time period 12
  Time period 13
  Time period 14
  Time period 15
  Time period 16
  Time period 17
  Time period 18
  Time period 19
  Time period 20
  Time period 21
  Time period 22
  Time period 23
  Time period 24
  Time period 25
  Time period 26
  Time period 27
  Time period 28
  Time period 29
  Time period 30
  Time period 31
  Time period 32
  Time period 33
  Time period 34
  Time period 35
Setting up SOE constraints for oil
  Time period 0
  Time period 1
  Time period 2
  Time period 3
  Time period 4
  Time period 5
  Time period 6
  Time period 7
  Time period 8
  Time period 9
  Time period 10
  Time period 11
  Time period 12
  Time period 13
  Time period 14
  Time period 15
  Time period 16
  Time period 17
  Time period 18
  Time period 

In [5]:
# Results Analysis and Visualization
model2_cost = []
for t in T:
    period_cost = sum(fuel_prices_df.iloc[t][fuel] * x[fuel, t].X for fuel in F)
    model2_cost.append(period_cost)

In [6]:
# Save results to csv
results_dir = os.path.join(current_dir.parent, "results", "model2")
os.makedirs(results_dir, exist_ok=True)
results_file = os.path.join(results_dir, "model2_results.csv")
import pandas as pd
model2_results_df = pd.DataFrame({
    'Month': demand_df.index,
    'Cost_EUR': model2_cost,
    'Accumulated_Cost': pd.Series(model2_cost).cumsum(),
    'Coal_Bought': [x['coal', t].X for t in T],
    'Oil_Bought': [x['oil', t].X for t in T],
    'Gas_Bought': [x['gas', t].X for t in T],
    'Coal_Generated': [y['coal', t].X for t in T],
    'Oil_Generated': [y['oil', t].X for t in T],
    'Gas_Generated': [y['gas', t].X for t in T],
    'Coal_Stored': [s['coal', t].X for t in T],
    'Oil_Stored': [s['oil', t].X for t in T],
    'Gas_Stored': [s['gas', t].X for t in T],
})
model2_results_df.to_csv(results_file, index=False)
print(f"Model 2 results saved to {results_file}")


Model 2 results saved to /Users/lalka/Projects/46750_g28_Assignment_2/results/model2/model2_results.csv
